In [1]:
import pandas as pd
import numpy as np
import statsmodels.regression.linear_model as sm
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tools.tools import add_constant

### Linear Regression

In [38]:
returns = pd.read_csv("/Users/luzhang/Documents/GitHub/WrappingUp_Exploring_Intraday_Momentum_2rd_try/Data/SPY_Returns_with_regime.csv")

In [39]:
returns = add_constant(returns)
returns

,const,datetime,R_LH,R_LH0,R_ROD,R_ON,R_FH,R_ONFH,R_M,R_SLH,regime
0,1.0,2007-06-15,-0.000783,-0.000785,0.003012,0.003471,-0.000065,0.003405,0.000131,-0.000522,0
1,1.0,2007-06-18,-0.000458,-0.000783,-0.000523,0.002025,-0.001043,0.000980,-0.000914,-0.000588,0
2,1.0,2007-07-10,-0.003500,-0.000914,-0.010843,-0.005422,-0.001511,-0.006924,-0.003420,-0.000528,0
3,1.0,2007-08-01,0.014601,-0.009672,-0.006121,-0.001307,0.000482,-0.000825,-0.005438,0.000138,1
4,1.0,2007-08-02,-0.002303,0.014601,0.006889,0.001023,0.003339,0.004365,-0.006180,0.008746,0
...,...,...,...,...,...,...,...,...,...,...,...
2768,1.0,2018-10-25,-0.004533,-0.007299,0.023188,0.008106,0.002618,0.010746,0.011975,0.000331,0
2769,1.0,2018-10-26,0.000189,-0.004533,-0.018139,-0.015622,0.000075,-0.015547,-0.005528,0.002911,1
2770,1.0,2018-10-29,0.004991,0.000189,-0.010554,0.013155,0.000781,0.013947,-0.016543,-0.007749,0
2771,1.0,2018-10-30,0.002639,0.004991,0.012528,-0.000493,0.012250,0.011751,-0.002286,0.003061,0


In [40]:
y = returns.R_LH

In [41]:
X = returns[["R_LH0","R_ROD","R_ON","R_FH","R_ONFH","R_M", "R_SLH"]]

In [42]:
# define a function find the features after backward Elimination
def BackwardEli(X,y):
    cols = list(X.columns)
    pmax = 1
    while (len(cols)>0):
        p = []
        X_1 = X[cols]
        X_1 = add_constant(X_1)
        model = sm.OLS(y,X_1).fit()
        p = pd.Series(model.pvalues.values[1:],index=cols)
        pmax = max(p)
        feature_with_p_max = p.idxmax()
        if(pmax>0.05):
            cols.remove(feature_with_p_max)
        else:
            break
    return cols

In [43]:
cols = BackwardEli(X,y)
final_input_X = add_constant(returns[cols])
final = sm.OLS(y,final_input_X).fit()
final.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   R_LH   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     40.72
Date:                Thu, 12 Oct 2023   Prob (F-statistic):           7.24e-55
Time:                        13:06:57   Log-Likelihood:                 11634.
No. Observations:                2773   AIC:                        -2.325e+04
Df Residuals:                    2765   BIC:                        -2.320e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4.281e-05   6.95e-05     -0.616      0.538      -0.000    9.35e-05
R_LH0         -0.1520      0.018     -8.272      0.000      -0.188      -0.116
R_ROD         -0.3314      0.122     -2.720      0.007      -0.570      -0.092
R_ON          -2.9958      0.930     -3.220      0.001      -4.820      -1.172
R_FH          -3.0970      0.923     -3.354      0.001      -4.907      -1.287
R_ONFH         3.4041      0.947      3.593      0.000       1.546       5.262
R_M            0.3732      0.122      3.057      0.002       0.134       0.613
R_SLH          0.5206      0.122      4.277      0.000       0.282       0.759
==============================================================================
Omnibus:                      839.443   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            44615.775
Skew:                          -0.625   Prob(JB):                         0.00
Kurtosis:                      22.611   Cond. No.                     2.33e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.33e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""